# Imports

In [11]:
import sys
sys.path.append('../')
from datetime import datetime
import numpy as np
import pandas as pd

from run_all.main_preprocess import load_data, add_features
from utilities.utilities import get_latest_file

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Preprocess

In [2]:
%%time
# Load original sources and combine to one DataFrame
df_dataset_WMO = load_data(region='gemeente')

Get 'WMO' tables
Number of tables to collect: 4
Pythonic iteration 2020 for table 84908NED
Pythonic iteration 2019 for table 84753NED
Pythonic iteration 2018 for table 84752NED
Pythonic iteration 2017 for table 84751NED
Get 'WIJK' tables
Number of tables to collect: 6
Pythonic iteration 2020 for table 84799NED
Pythonic iteration 2019 for table 84583NED
Pythonic iteration 2018 for table 84286NED
Pythonic iteration 2017 for table 83765NED
Pythonic iteration 2016 for table 83487NED
Pythonic iteration 2015 for table 83220NED
Get 'Huishoudens' data
Number of tables to collect: 1
Pythonic iteration all for table 83504NED
Get 'Bevolkings' data
Number of tables to collect: 1
Pythonic iteration all for table 70072NED
Get 'Gemeentelijke heffingen' data
Number of tables to collect: 1
Pythonic iteration all for table 83643NED
Combine all datasets to one DataFrame
CPU times: user 48.9 s, sys: 10.3 s, total: 59.2 s
Wall time: 9min 22s


In [ ]:
# datapath = '../../data/'
# filename = 'df_WMO_WIJK_HOUSEHOLDS_POP_LEVY.parquet.gzip'

# df_dataset_WMO.to_parquet(datapath + filename,
#               compression='gzip')

# df_dataset_WMO = pd.read_parquet(datapath + filename)

In [3]:
# 21:44
df_dataset_WMO.head()

perioden  typemaatwerkarrangement  wmoclienten  wmoclientenper1000inwoners  periodennum aantalinkomensontvangers  aantalinwoners actieven1575jaar afstandtotgrotesupermarkt afstandtothuisartsenpraktijk afstandtotkinderdagverblijf afstandtotschool alandbouwbosbouwenvisserij  bedrijfsmotorvoertuigen bedrijfsvestigingentotaal  bevolkingsdichtheid bfnijverheidenenergie bouwjaarvanaf2000 bouwjaarvoor2000  dekkingspercentage  eenpersoonshuishoudens eigendomonbekend geboorterelatief geboortetotaal  gehuwd  gemeentenaam gemelectriciteitsverbruikappartement gemelectriciteitsverbruikeigenwoning gemelectriciteitsverbruikhoekwoning gemelectriciteitsverbruikhuurwoning gemelectriciteitsverbruiktussenwoning gemelectriciteitsverbruiktweeondereenkapwoning gemelectriciteitsverbruikvrijstaandewoning gemgasverbruikappartement gemgasverbruikeigenwoning gemgasverbruikhoekwoning gemgasverbruikhuurwoning gemgasverbruiktussenwoning gemgasverbruiktweeondereenkapwoning  \
codering_regio interval                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
GM0003         2019      1e halfjaar 2019  Hulp bij het huishouden        505.0                        43.0       201901                     None           11721             None                       0.9                          1.4                         0.9              0.9                         35                      NaN                       585                493.0                    80                17               83                 NaN                    2097                0                7             79    4874    Appingedam                                 1760                                 2690                                2570                                2130                                  2460                                           2520                                       2990                       900                      1540                     1470                     1140                       1290                                1410   
               2020      1e halfjaar 2020  Hulp bij het huishouden        535.0                        46.0       202001                     None           11642             None                      None                         None                        None             None                       None                      NaN                      None                490.0                  None              None             None                 NaN                    2094             None             None           None    4795    Appingedam                                 None                                 None                                None                                None                                  None                                           None                                       None                      None                      None                     None                     None                       None                                None   
GM0007         2017      1e halfjaar 2017  Hulp bij het huishoude

In [4]:
df_dataset_WMO.shape

(2252, 459)

In [5]:
list(df_dataset_WMO.columns)

['perioden',
 'typemaatwerkarrangement',
 'wmoclienten',
 'wmoclientenper1000inwoners',
 'periodennum',
 'aantalinkomensontvangers',
 'aantalinwoners',
 'actieven1575jaar',
 'afstandtotgrotesupermarkt',
 'afstandtothuisartsenpraktijk',
 'afstandtotkinderdagverblijf',
 'afstandtotschool',
 'alandbouwbosbouwenvisserij',
 'bedrijfsmotorvoertuigen',
 'bedrijfsvestigingentotaal',
 'bevolkingsdichtheid',
 'bfnijverheidenenergie',
 'bouwjaarvanaf2000',
 'bouwjaarvoor2000',
 'dekkingspercentage',
 'eenpersoonshuishoudens',
 'eigendomonbekend',
 'geboorterelatief',
 'geboortetotaal',
 'gehuwd',
 'gemeentenaam',
 'gemelectriciteitsverbruikappartement',
 'gemelectriciteitsverbruikeigenwoning',
 'gemelectriciteitsverbruikhoekwoning',
 'gemelectriciteitsverbruikhuurwoning',
 'gemelectriciteitsverbruiktussenwoning',
 'gemelectriciteitsverbruiktweeondereenkapwoning',
 'gemelectriciteitsverbruikvrijstaandewoning',
 'gemgasverbruikappartement',
 'gemgasverbruikeigenwoning',
 'gemgasverbruikhoekwoning',

In [6]:
drop_list = ['popcodea',
 'popcodeb',
 'popcodec',
 'popcoded',
 'popcodee',
 'popcodef',
 'popcodeg',
 'popcodeh',
 'popcodei',
 'popcodej',
 'popcodek',
 'popcodel',
 'popcodem',
 'popcoden',
 'popcodeo',
 'popcodep',
 'popcodeq',
 'popcoder',
 'popnaama',
 'popnaamb',
 'popnaamc',
 'popnaamd',
 'popnaame',
 'popnaamf',
 'popnaamg',
 'popnaamh',
 'popnaami',
 'popnaamj',
 'popnaamk',
 'popnaaml',
 'popnaamm',
 'popnaamn',
 'popnaamo',
 'popnaamp',
 'popnaamq',
 'popnaamr',
'popkoppelvariabeleregiocode']

In [7]:
df_dataset_WMO = df_dataset_WMO.drop(drop_list, axis=1)

In [16]:
df_dataset_WMO.head()

perioden  typemaatwerkarrangement  wmoclienten  wmoclientenper1000inwoners  periodennum  aantalinkomensontvangers  aantalinwoners  actieven1575jaar  afstandtotgrotesupermarkt  afstandtothuisartsenpraktijk  afstandtotkinderdagverblijf  afstandtotschool  alandbouwbosbouwenvisserij  bedrijfsmotorvoertuigen  bedrijfsvestigingentotaal  bevolkingsdichtheid  bfnijverheidenenergie  bouwjaarvanaf2000  bouwjaarvoor2000  dekkingspercentage  eenpersoonshuishoudens  eigendomonbekend  geboorterelatief  geboortetotaal  gehuwd  gemeentenaam  gemelectriciteitsverbruikappartement  gemelectriciteitsverbruikeigenwoning  gemelectriciteitsverbruikhoekwoning  gemelectriciteitsverbruikhuurwoning  gemelectriciteitsverbruiktussenwoning  gemelectriciteitsverbruiktweeondereenkapwoning  gemelectriciteitsverbruikvrijstaandewoning  gemgasverbruikappartement  gemgasverbruikeigenwoning  gemgasverbruikhoekwoning  gemgasverbruikhuurwoning  gemgasverbruiktussenwoning  \
codering_regio interval                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
GM0003         2019      1e halfjaar 2019  Hulp bij het huishouden        505.0                        43.0       201901                       NaN           11721               NaN                        0.9                           1.4                          0.9               0.9                        35.0                      NaN                      585.0                493.0                   80.0               17.0              83.0                 NaN                    2097               0.0               7.0            79.0    4874    Appingedam                                1760.0                                2690.0                               2570.0                               2130.0                                 2460.0                                          2520.0                                      2990.0                      900.0                     1540.0                    1470.0                    1140.0                      1290.0   
               2020      1e halfjaar 2020  Hulp bij het huishouden        535.0                        46.0       202001                       NaN           11642               NaN                        NaN                           NaN                          NaN               NaN                         NaN                      NaN                        NaN                490.0                    NaN                NaN               NaN                 NaN                    2094               NaN               NaN             NaN    4795    Appingedam                                   NaN                                   NaN                                  NaN                                  NaN                                    NaN                                             NaN                                         NaN                        NaN                        NaN                       NaN                       NaN                         NaN   
GM0007         2017      1e halfjaar 2017  Hulp bij het huishouden        320.0                        36

In [12]:
df_dataset_WMO.fillna(value=np.nan, inplace=True)

In [14]:
%%time
# Feature engineering to get more features
df_dataset_WMO_with_features = add_features(df_dataset_WMO)

CPU times: user 223 ms, sys: 54.7 ms, total: 278 ms
Wall time: 290 ms


/usr/local/Caskroom/miniconda/base/envs/jads-env/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/usr/local/Caskroom/miniconda/base/envs/jads-env/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [15]:
df_dataset_WMO_with_features.shape

(2252, 457)

In [ ]:
datapath = '../../data/'
filename = 'df_WMO_WIJK_HOUSEHOLDS_POP_LEVY_features.parquet.gzip'

df_dataset_WMO_with_features.to_parquet(datapath + filename,
              compression='gzip')

# df_dataset_WMO = pd.read_parquet(datapath + filename)

## Optional: Write temporary result

In [ ]:
# suffix_datetime = datetime.strftime(datetime.now(), format='%Y%m%d%H%M')

# df_dataset_WMO_with_features.to_parquet(f'../../data/df_preprocess_WMO_{suffix_datetime}.parquet.gzip',
#               compression='gzip')

# Train

## Optional: Load previous dataset

In [ ]:
# ## HARDCODED
# datapath = '../../data/'
# filename = 'df_preprocess_WMO_202103211137.parquet.gzip'
# df = pd.read_parquet(datapath + filename)

# ## SELECT LAST FILE
# datapath = '../../data/'
# df = get_latest_file(filename_str_contains='df_preprocess_WMO_', datapath=datapath, filetype='parquet')

## Train model
Code will be added later